In [1]:
import torch
import yfinance as yf
import pandas as pd
import os
# import numpy as np
import requests
import datetime
import ray
from ray import tune, air

import sys
sys.path.append('./data')
sys.path.append('./training')
import data, training


torch.cuda.is_available()

True

In [2]:
# import os
# import torch
# import torch.nn as nn
# import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split as TTS
# import ray
# from torch.nn.parallel import DistributedDataParallel as DDP
# import torch.distributed as dist
# import torch.multiprocessing as mp
# import wandb
# from tqdm.auto import tqdm, trange
# from ray.air import session


# from ray.train.torch import TorchCheckpoint
# from ray.train import Trainer

# # from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
# from ray.tune.search.bohb import TuneBOHB
# from ray.tune.schedulers import ASHAScheduler
# from ray.air.callbacks.wandb import WandbLoggerCallback
# from functools import partial

# import math

In [3]:
BATCH_SIZE=96
WINDOW_SIZE=32
NUM_WORKERS=6
N_FEATURES=1
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
ray.init(num_cpus=int(os.cpu_count() * .75), num_gpus=1)

2023-04-12 10:00:45,506	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.6
Ray version:,2.0.1
Dashboard:,http://127.0.0.1:8265


In [5]:
# Need to go requests route to avoid forbidden error
url = 'https://stockanalysis.com/list/biggest-companies/'

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url, headers=header)

dfs = pd.read_html(r.text)

symbols = list(dfs[0]['Symbol'].iloc[:25].values)

(raylet) /home/alexander/miniconda3/envs/hyperband/lib/python3.10/site-packages/ray/dashboard/agent.py:470: DeprecationWarning: There is no current event loop
(raylet)   loop = asyncio.get_event_loop()


In [6]:
# Maximum number of days for YFinance 5m data is 60, but it only takes 59 day diff
end_date = datetime.date.today() 
start_date = end_date - datetime.timedelta(days=59)

dataset = yf.download(
    symbols,
    period='max', 
    interval='5m',
    start=start_date, 
    end=end_date
)['Adj Close']

train, val = data.get_train_val_dicts(dataset)

[*********************100%***********************]  25 of 25 completed

1 Failed download:
- BRK.B: No timezone found, symbol may be delisted


In [7]:
config = {
    'device': DEVICE,
    'D_MODEL': tune.choice([32, 64]),
    'N_HEADS': tune.choice([2, 8]),
    'D_FF': tune.choice([128, 256]),
    'DROPOUT': tune.choice([0., .05, .1, .15]),
    'ACTIVATION': tune.choice(['relu', 'gelu']),
    'NUM_LAYERS': tune.choice([3, 5]),
    'LR': tune.choice([1e-3, 1e-4, 1e-5]),
    'WINDOW_SIZE': tune.choice([32, 64])
}
param_space = {'train_loop_config': config}

In [8]:
trainer, tune_config = training.setup_ray(train, val, BATCH_SIZE, param_space, n_tries=8)

tuner = tune.Tuner(
    trainer,
    run_config=air.RunConfig(
        local_dir="./tfmr_logger/",
    ),
    tune_config=tune_config,
)

result = tuner.fit()

2023-04-12 10:00:53,098	WARNING bohb_search.py:152 -- You passed a `space` parameter to <class 'ray.tune.search.bohb.bohb_search.TuneBOHB'> that contained unresolved search space definitions. <class 'ray.tune.search.bohb.bohb_search.TuneBOHB'> should however be instantiated with fully configured search spaces only. To use Ray Tune's automatic search space conversion, pass the space definition as part of the `param_space` argument to `tune.Tuner()` instead.


Trial name,status,loc,train_loop_config...,train_loop_config...,train_loop_config...,train_loop_config...,train_loop_config/LR,train_loop_config...,train_loop_config...,train_loop_config...,iter,total time (s),train_loss,val_loss,_timestamp
TorchTrainer_ad153ce4,RUNNING,10.0.0.153:53910,gelu,0.1,128,64,1e-05,5,8,64,,,,,
TorchTrainer_7027d7ec,PAUSED,10.0.0.153:52947,relu,0.05,128,64,1e-05,5,2,64,3,15.2203,136.887,133.332,1681308071
TorchTrainer_7236c520,PAUSED,10.0.0.153:53104,gelu,0,256,32,1e-05,5,8,32,3,17.8565,56.0579,54.2382,1681308093
TorchTrainer_7b9a2e36,PAUSED,10.0.0.153:53267,gelu,0.15,128,64,1e-05,5,8,64,3,18.6535,136.899,133.398,1681308116
TorchTrainer_88b787a8,PAUSED,10.0.0.153:53428,gelu,0.15,128,32,0.0001,3,8,32,3,16.7603,56.4745,54.5568,1681308136
TorchTrainer_95d3f296,PAUSED,10.0.0.153:53595,gelu,0.15,256,32,0.001,5,2,64,3,14.4136,137.273,133.83,1681308154
TorchTrainer_a25971b2,PAUSED,10.0.0.153:53750,relu,0.15,128,64,1e-05,3,2,64,3,15.062,137.319,134.001,1681308173
TorchTrainer_b867bf86,PENDING,,gelu,0.1,128,64,0.001,3,8,32,,,,,


(RayTrainWorker pid=52986) 2023-04-12 10:00:58,958	INFO config.py:71 -- Setting up process group for: env:// [rank=0, world_size=1]
(RayTrainWorker pid=52986) wandb: wandb version 0.14.2 is available!  To upgrade, please run:
(RayTrainWorker pid=52986) wandb:  $ pip install wandb --upgrade
(RayTrainWorker pid=52986) wandb: Tracking run with wandb version 0.13.4
(RayTrainWorker pid=52986) wandb: Run data is saved locally in /media/alexander/38F66E50F66E0E86/Users/Alexander/Downloads/Python Projects/medium_articles/tfmr_logger/TorchTrainer_2023-04-12_10-00-53/TorchTrainer_7027d7ec_1_ACTIVATION=relu,DROPOUT=0.0500,D_FF=128,D_MODEL=64,LR=0.0000,NUM_LAYERS=5,N_HEADS=2,WINDOW_SIZE=64_2023-04-12_10-00-53/rank_0/wandb/run-20230412_100101-2qql1mm9
(RayTrainWorker pid=52986) wandb: Run `wandb offline` to turn off syncing.
(RayTrainWorker pid=52986) wandb: Syncing run stoic-brook-63
(RayTrainWorker pid=52986) wandb: ⭐️ View project at https://wandb.ai/lavellea/torch_stock_transformer
(RayTrainWor

Result for TorchTrainer_7027d7ec:
  _time_this_iter_s: 9.664819955825806
  _timestamp: 1681308069
  _training_iteration: 1
  date: 2023-04-12_10-01-09
  done: false
  experiment_id: 366d518997094ed6b511f5dff8dab268
  hostname: Ajax
  iterations_since_restore: 1
  node_ip: 10.0.0.153
  pid: 52947
  should_checkpoint: true
  time_since_restore: 13.066022157669067
  time_this_iter_s: 13.066022157669067
  time_total_s: 13.066022157669067
  timestamp: 1681308069
  timesteps_since_restore: 0
  train_loss: 138.00783746892756
  training_iteration: 1
  trial_id: 7027d7ec
  val_loss: 133.90579223632812
  warmup_time: 0.0022439956665039062
  


100%|██████████| 11/11 [00:00<00:00, 28.81it/s]
(RayTrainWorker pid=52986) /home/alexander/miniconda3/envs/hyperband/lib/python3.10/site-packages/torch/nn/modules/transformer.py:569: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at ../aten/src/ATen/native/transformers/attention.cpp:150.)
(RayTrainWorker pid=52986)   return torch._transformer_encoder_layer_fwd(
 82%|████████▏ | 9/11 [00:00<00:00, 23.69it/s]
(RayTrainWorker pid=52986) Exception in thread MsgRouterThr:
(RayTrainWorker pid=52986) Traceback (most recent call last):
(RayTrainWorker pid=52986)   File "/home/alexander/miniconda3/envs/hyperband/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
(RayTrainWorker pid=52986)     self.run()
(RayTrainWorker pid=52986)   File "/home/alexander/miniconda3/envs/hyperband/lib/python3.10/threading.py", line 953, in run
(RayTrainWorker pid=52986)     self._target(

...

Result for TorchTrainer_95d3f296:
  _time_this_iter_s: 9.143475770950317
  _timestamp: 1681308152
  _training_iteration: 1
  date: 2023-04-12_10-02-33
  done: false
  experiment_id: 89c0a50e22834b81ae75a24bb05f1e1f
  hostname: Ajax
  iterations_since_restore: 1
  node_ip: 10.0.0.153
  pid: 53595
  should_checkpoint: true
  time_since_restore: 12.466220617294312
  time_this_iter_s: 12.466220617294312
  time_total_s: 12.466220617294312
  timestamp: 1681308153
  timesteps_since_restore: 0
  train_loss: 139.24953668767756
  training_iteration: 1
  trial_id: 95d3f296
  val_loss: 134.23497009277344
  warmup_time: 0.0022466182708740234
  


100%|██████████| 11/11 [00:00<00:00, 34.40it/s]
(RayTrainWorker pid=53635) /home/alexander/miniconda3/envs/hyperband/lib/python3.10/site-packages/torch/nn/modules/transformer.py:569: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at ../aten/src/ATen/native/transformers/attention.cpp:150.)
(RayTrainWorker pid=53635)   return torch._transformer_encoder_layer_fwd(
100%|██████████| 11/11 [00:00<00:00, 34.29it/s]
(RayTrainWorker pid=53635) /home/alexander/miniconda3/envs/hyperband/lib/python3.10/site-packages/torch/nn/modules/transformer.py:569: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at ../aten/src/ATen/native/transformers/attention.cpp:150.)
(RayTrainWorker pid=53635)   return torch._transformer_encoder_layer_fwd(
(RayTrainWorker pid=53635) Exception

Result for TorchTrainer_a25971b2:
  _time_this_iter_s: 9.635966062545776
  _timestamp: 1681308171
  _training_iteration: 1
  date: 2023-04-12_10-02-51
  done: false
  experiment_id: 8b9e82109f594add8a2d0601e26241df
  hostname: Ajax
  iterations_since_restore: 1
  node_ip: 10.0.0.153
  pid: 53750
  should_checkpoint: true
  time_since_restore: 12.996700763702393
  time_this_iter_s: 12.996700763702393
  time_total_s: 12.996700763702393
  timestamp: 1681308171
  timesteps_since_restore: 0
  train_loss: 139.89222994717684
  training_iteration: 1
  trial_id: a25971b2
  val_loss: 135.53860092163086
  warmup_time: 0.002370595932006836
  


100%|██████████| 11/11 [00:00<00:00, 36.63it/s]
(RayTrainWorker pid=53790) /home/alexander/miniconda3/envs/hyperband/lib/python3.10/site-packages/torch/nn/modules/transformer.py:569: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at ../aten/src/ATen/native/transformers/attention.cpp:150.)
(RayTrainWorker pid=53790)   return torch._transformer_encoder_layer_fwd(
100%|██████████| 11/11 [00:00<00:00, 34.99it/s]
(RayTrainWorker pid=53790) /home/alexander/miniconda3/envs/hyperband/lib/python3.10/site-packages/torch/nn/modules/transformer.py:569: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at ../aten/src/ATen/native/transformers/attention.cpp:150.)
(RayTrainWorker pid=53790)   return torch._transformer_encoder_layer_fwd(
(RayTrainWorker pid=53790) Exception

Result for TorchTrainer_ad153ce4:
  _time_this_iter_s: 9.941187620162964
  _timestamp: 1681308190
  _training_iteration: 1
  date: 2023-04-12_10-03-11
  done: false
  experiment_id: 0f4b64c9d0384227be155ae00a01a646
  hostname: Ajax
  iterations_since_restore: 1
  node_ip: 10.0.0.153
  pid: 53910
  should_checkpoint: true
  time_since_restore: 13.430606365203857
  time_this_iter_s: 13.430606365203857
  time_total_s: 13.430606365203857
  timestamp: 1681308191
  timesteps_since_restore: 0
  train_loss: 138.9628046209162
  training_iteration: 1
  trial_id: ad153ce4
  val_loss: 134.1124382019043
  warmup_time: 0.0025713443756103516
  
